## Setup

Please execute the cell(s) below to initialize the notebook environment.

In [9]:
# @title Install dependencies
# !pip install poetry

In [10]:
# @title Install SMorph Python module
# !pip install https://github.com/swanandlab/SMorph/releases/download/v0.1.1-alpha/SMorph-0.1.1.tar.gz

In [1]:
import napari


In [5]:
import numpy as np
im = np.load('Cache/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi.npy')
viewer = napari.view_image(im)

In [1]:
# Imports
on_colab = 'google.colab' in str(get_ipython())

import warnings
warnings.filterwarnings('ignore')

import json
from os import getcwd, path 

if not on_colab:
  import napari
import smorph as sm
import smorph.util.autocrop as ac
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

import PyQt5
import superqt
from vispy.geometry.rect import Rect
from smorph.util.autocrop._postprocessing import _segment_clump

from magicgui import magicgui

In [2]:
# Helper function
def view_3D(*args):
    """Views a 3D image."""
    n_images = len(args)
    if not on_colab and n_images > 0:
        with napari.gui_qt():
            viewer = napari.view_image(**args[0], ndisplay=3, scale=pipe.SCALE)
            for itr in range(1, n_images):
                if args[itr]['data'].max() > 1 and args[itr]['data'].max() % 1 == 0:
                    if 'colormap' in args[itr].keys():
                        del args[itr]['colormap']
                    if 'gamma' in args[itr].keys():
                        del args[itr]['gamma']
                    viewer.add_labels(**args[itr], scale=pipe.SCALE)
                else:
                    viewer.add_image(**args[itr], scale=pipe.SCALE)

---

---
## Step 0: Import Reference image

---

## Step 1: Import Microscopic Image of the Tissue

Set `TISSUE_IMAGE` to the path of the image file to be processed.

1. Deconvolution
2. Rolling ball background subtraction
3. CLAHE
4. ROI selection
5. Non-local means denoising using auto-calibrated parameters using J-Invariance

In [3]:
TISSUE_IMAGE = 'Datasets/Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi.czi'  #@param
FILE_ROI = 'Datasets/Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi_ML.roi'
NAME_ROI = 'ML'

DECONV_ITR = 30
CLIP_LIMIT = .02

REF_IMAGE = 'Datasets/Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP3.1M_1_SINGLE MARK_20X_SEC 1_RIGHT HILUS_28 DAYczi.czi'
REF_FILE_ROI = 'Datasets/Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP3.1M_1_SINGLE MARK_20X_SEC 1_RIGHT HILUS_28 DAYczi_ML.roi'

cache_dir = 'Cache/'

pipe = ac.TissueImage(TISSUE_IMAGE, FILE_ROI, NAME_ROI, DECONV_ITR, CLIP_LIMIT, REF_IMAGE, REF_FILE_ROI, cache_dir)

---

## Step 2: Select ROI using Polygonal Lasso Tool

Set two variables:
- `SELECT_ROI`: True, If you want to select ROI manually; else False
- `NAME_ROI`: Name of the manually selected ROI
- `FILE_ROI`: Path to the ROI file; else None

In [4]:
view_3D({'data': pipe.imdeconvolved, 'colormap': 'magma', 'name': 'deconvolved'},
        {'data': pipe.impreprocessed, 'colormap': 'magma', 'name': 'preprocessed'},
        {'data': pipe.imdenoised, 'colormap': 'magma', 'name': 'denoised'})

---

## Step 3: Segmentation

### 3.1 Global threshold & color label cells

Set two parameters:
- `LOW_THRESH`: Pixel intensity value corresponding to faintest branch's edge
- `HIGH_THRESH`: Pixel intensity value corresponding to faintest soma

Understand their effect by configuring three parameters:
- `LOW_DELTA`: Pixel intensity value corresponding to change in `LOW_THRESH`
- `HIGH_DELTA`: Pixel intensity value corresponding to change in `HIGH_THRESH`
- `N_STEPS`: Number of steps of delta in threshold to take in both directions

In [8]:
LOW_THRESH = .07  # .10 #.09
HIGH_THRESH = .12

LOW_DELTA = .01
HIGH_DELTA = .01
N_STEPS = 0
results = None
import skimage
%matplotlib inline
def test_thresholds(low_thresh_init, low_thresh, high_thresh_init,
                    high_thresh, low_delta, high_delta, n_steps):
  global results, LOW_THRESH, HIGH_THRESH, LOW_DELTA, HIGH_DELTA, N_STEPS
  LOW_THRESH, HIGH_THRESH, N_STEPS = low_thresh, high_thresh, n_steps
  if low_thresh_init is not None:
    LOW_THRESH = eval(f'skimage.filters.threshold_{low_thresh_init}(pipe.imdenoised[:, lly:ury, llx:urx])')
  if high_thresh_init is not None:
    if high_thresh_init == 'isodata':
      HIGH_THRESH = eval(f'skimage.filters.threshold_{high_thresh_init}(pipe.imdenoised[:, lly:ury, llx:urx])')
  LOW_DELTA, HIGH_DELTA = low_delta, high_delta
  results = ac.testThresholds(pipe.imdenoised, LOW_THRESH, HIGH_THRESH, LOW_DELTA,
                              HIGH_DELTA, N_STEPS, 'gist_earth')

if on_colab:
  _ = widgets.interact(test_thresholds,
                      low_thresh_init=[None, *sm.util.THRESHOLD_METHODS],
                      low_thresh=widgets.FloatSlider(LOW_THRESH, min=0, max=1, step=.01,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      high_thresh_init=[None, *sm.util.THRESHOLD_METHODS],
                      high_thresh=widgets.FloatSlider(HIGH_THRESH, min=0, max=1, step=.01,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      low_delta=widgets.FloatSlider(LOW_DELTA, min=0, max=1, step=.0005,
                                                    readout_format='.4f', layout=widgets.Layout(width='100%')),
                      high_delta=widgets.FloatSlider(HIGH_DELTA, min=0, max=1, step=.0005,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      n_steps=widgets.IntSlider(N_STEPS, min=0, max=10,
                                                layout=widgets.Layout(width='100%'))
  )
  # view_3D({'data': pipe.imdenoised, 'colormap': 'gray_r', 'name': 'denoised'}, *results)
else:
  viewer = napari.Viewer(ndisplay=3)
  viewer.add_image(pipe.impreprocessed, scale=pipe.SCALE)
  @magicgui(
    call_button="Test Thresholds",
    low_auto_thresh={'choices': [None, *sm.util.THRESHOLD_METHODS]},
    low_thresh={"widget_type": "FloatSlider", 'max': 1},
    high_auto_thresh={'choices': [None, *sm.util.THRESHOLD_METHODS]},
    high_thresh={"widget_type": "FloatSlider", 'max': 1},
    low_delta={"widget_type": "FloatSlider", 'max': 1},
    high_delta={"widget_type": "FloatSlider", 'max': 1}
  )
  def test_thresholds_gui(
      low_auto_thresh=None,
      low_thresh=LOW_THRESH,
      high_auto_thresh=None,
      high_thresh=HIGH_THRESH,
      low_delta=LOW_DELTA,
      high_delta=HIGH_DELTA,
      n_steps=N_STEPS
  ):
    global results, LOW_THRESH, HIGH_THRESH, LOW_DELTA, HIGH_DELTA, N_STEPS
    LOW_THRESH, HIGH_THRESH, N_STEPS = low_thresh, high_thresh, n_steps
    lly, llx, ury, urx = pipe.in_box
    if low_auto_thresh is not None:
      LOW_THRESH = eval(f'skimage.filters.threshold_{low_auto_thresh}(pipe.imdenoised[:, lly:ury, llx:urx])')
    if high_auto_thresh is not None:
      # if high_auto_init == 'isodata':
      HIGH_THRESH = eval(f'skimage.filters.threshold_{high_auto_thresh}(pipe.imdenoised[:, lly:ury, llx:urx])')
    LOW_DELTA, HIGH_DELTA = low_delta, high_delta
    results = ac.core._testThresholds(pipe.imdenoised, LOW_THRESH, HIGH_THRESH, LOW_DELTA,
                                      HIGH_DELTA, N_STEPS)
    past_state = viewer.window.qt_viewer.view.camera.get_state()
    viewer.layers.clear()

    viewer.add_image(pipe.impreprocessed, scale=pipe.SCALE)

    n_images = len(results)
    for itr in range(n_images):
      viewer.add_labels(**results[itr], scale=pipe.SCALE)
    viewer.window.qt_viewer.view.camera.set_state(past_state)

  viewer.window.add_dock_widget(test_thresholds_gui)
  test_thresholds_gui()

In [5]:
print(LOW_THRESH, HIGH_THRESH)

pipe.segment(LOW_THRESH, HIGH_THRESH)

0.07 0.12
Prefiltering Volume: 254518


In [17]:
view_3D({'data': pipe.imsegmented, 'colormap': 'inferno', 'name': 'segmented'},
        #{'data': pipe.filtered_labels, 'colormap': 'gray', 'gamma': .8, 'name': 'filtered_labels'},
        {'data': pipe.labels, 'colormap': 'gist_earth', 'gamma': .8, 'name': 'labels'})

### 3.3 Visualize segmented cells to determine cutoff volumes

#### 3.3.1 Check segmented cells on whole image

In [6]:
pipe.volume_cutoff()

In [7]:
somas_load_path = 'Autocropped/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi-ML/.somas_estimates.npy'
pipe.approximate_somas(src=somas_load_path)

[array([ 19, 879, 430], dtype=int64),
 array([  5, 695, 205], dtype=int64),
 array([ 13, 705, 210], dtype=int64),
 array([ 10, 658, 303], dtype=int64),
 array([ 23, 397,  74], dtype=int64),
 array([ 33, 413,  47], dtype=int64),
 array([  8, 729, 324], dtype=int64),
 array([  9, 523, 106], dtype=int64),
 array([ 10, 512, 119], dtype=int64),
 array([ 31, 226, 255], dtype=int64),
 array([ 36, 232, 236], dtype=int64),
 array([ 36, 236, 248], dtype=int64),
 array([  9, 111, 260], dtype=int64),
 array([ 15, 104, 259], dtype=int64),
 array([ 13, 230,  63], dtype=int64),
 array([ 13, 233,  80], dtype=int64),
 array([ 15, 237,  97], dtype=int64),
 array([ 29, 950, 372], dtype=int64),
 array([ 29, 959, 378], dtype=int64),
 array([  9, 737, 121], dtype=int64),
 array([ 32, 372, 295], dtype=int64),
 array([  10, 1001,  147], dtype=int64),
 array([  4, 993, 449], dtype=int64),
 array([ 19, 760, 162], dtype=int64),
 array([  5, 907, 197], dtype=int64),
 array([ 15, 925, 179], dtype=int64),
 array([ 

# napari clump sep

In [ ]:
# np.save('tmp_filtered_coords_CTL.npy', viewer.layers[-1].data)

In [ ]:
# Reproduce (reload)
# somas_estimates = np.vstack((somas_estimates, np.load('rejected_clumps-ML.npy')))

In [8]:
seed_src = 'Autocropped/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi-ML/.somas_estimates.npy'
pipe.separate_clumps(seed_src)

In [9]:
len(pipe.regions)

65

#### 3.3.2: Check batches of objects

In [10]:
pipe.show_segmented('grid', 50)

65 objects detected.
There will be 2 batches, set `BATCH_NO` from 0 to 1 inclusive


interactive(children=(IntSlider(value=0, description='BATCH_NO', layout=Layout(width='100%'), max=1), Output()…

#### 3.3.2: Check individual objects
Select individual objects using `OBJ_INDEX`.

In [11]:
pipe.FINAL_PT_ESTIMATES

In [12]:
pipe.show_segmented()

TypeError: '<=' not supported between instances of 'int' and 'NoneType'

In [ ]:
# np.save('tmp_filtered_coords_CTL.npy', viewer.layers[-1].data)
# reconstructed_labels = np.zeros(denoised.shape, dtype=int)
# for itr in range(len(regions)):
#     minz, miny, minx, maxz, maxy, maxx = regions[itr]['bbox']
#     reconstructed_labels[minz:maxz, miny:maxy, minx:maxx] += regions[itr]['image'] * (itr + 1)
# segmented = denoised * (reconstructed_labels > 0)
# len(regions)

---

## Step 4: Export autocropped 3D cells or 2D max intensity projections

Set two parameters:
- `LOW_VOLUME_CUTOFF`: to filter out noise/artifacts
- `HIGH_VOLUME_CUTOFF`: to filter out cell clusters

For choosing between 3D segmented cells or 2D max intensity projections:
- Set `OUTPUT_OPTION` = '3d' for 3D cells, or
- Set `OUTPUT_OPTION` = 'mip' for Max Intensity Projections.

In [ ]:
LOW_VOLUME_CUTOFF = regions[0]['vol']  # filter noise/artifacts
HIGH_VOLUME_CUTOFF = regions[-1]['vol']  # filter cell clusters
OUTPUT_OPTION = 'both'  # '3d' for 3D cells, 'mip' for Max Intensity Projections
SEGMENT_TYPE = 'both'
reconstructed_cells = None


def volume_range(low_volume_cutoff=LOW_VOLUME_CUTOFF,
                 high_volume_cutoff=HIGH_VOLUME_CUTOFF, output_option=OUTPUT_OPTION,
                 segment_type=SEGMENT_TYPE):
  global LOW_VOLUME_CUTOFF, HIGH_VOLUME_CUTOFF, OUTPUT_OPTION, SEGMENT_TYPE, reconstructed_cells
  LOW_VOLUME_CUTOFF, HIGH_VOLUME_CUTOFF = low_volume_cutoff, high_volume_cutoff
  OUTPUT_OPTION, SEGMENT_TYPE = output_option, segment_type

  reconstructed_cells = np.zeros_like(denoised)
  for region in regions:
    if LOW_VOLUME_CUTOFF <= region['vol'] <= HIGH_VOLUME_CUTOFF:
      minz, miny, minx, maxz, maxy, maxx = region['bbox']
      reconstructed_cells[minz:maxz, miny:maxy, minx:maxx] += region['image'] * denoised[minz:maxz, miny:maxy, minx:maxx]
  ac.projectXYZ(reconstructed_cells, .5, .5, 1, 'gist_heat')

_ = widgets.interact(volume_range, low_volume_cutoff=widgets.IntSlider(value=LOW_VOLUME_CUTOFF,
                         min=regions[0]['vol'], max=regions[-1]['vol'], layout=widgets.Layout(width='100%')),
                     high_volume_cutoff=widgets.IntSlider(value=HIGH_VOLUME_CUTOFF,
                         min=regions[0]['vol'], max=regions[-1]['vol'], layout=widgets.Layout(width='100%')),
                     output_option=['3d', 'mip', 'both'],
                     segment_type=['segmented', 'unsegmented', 'both'])
# view_3D({'data': original, 'colormap': 'gray', 'name': 'original'},
#         {'data': reconstructed_cells, 'name': 'output'})

In [ ]:
LOW_VOLUME_CUTOFF = regions[0]['vol']  # filter noise/artifacts
HIGH_VOLUME_CUTOFF = regions[-1]['vol']  # filter cell clusters
OUTPUT_OPTION = 'both'  # '3d' for 3D cells, 'mip' for Max Intensity Projections
SEGMENT_TYPE = 'both'
reconstructed_cells = None

viewer = napari.Viewer(ndisplay=3)
reconstructed_cells = np.zeros_like(denoised)
viewer.add_image(reconstructed_cells, colormap='inferno', scale=SCALE)
minz, miny, minx, maxz, maxy, maxx = region['bbox']


region_props = PyQt5.QtWidgets.QLabel()
region_inclusivity = np.ones(len(regions), dtype=bool)
REGION_INCLUSIVITY_LABELS = ['Include Region', 'Exclude Region']

n_region = 0
PROPS = ['area',
# 'convex_area',
# 'equivalent_diameter',
# 'euler_number',
# 'extent',
# 'feret_diameter_max',
# 'major_axis_length',
# 'minor_axis_length',
# 'solidity'
]

vol_cutoff_slider = superqt.QLabeledRangeSlider()
vol_cutoff_slider.setRange(0, regions[-1]['vol'])
vol_cutoff_slider.setOrientation(1)
vol_cutoff_slider.setValue([LOW_VOLUME_CUTOFF, HIGH_VOLUME_CUTOFF])
vol_cutoff_slider.setEdgeLabelMode(superqt.sliders._labeled.EdgeLabelMode.NoLabel)
vol_cutoff_slider.setContentsMargins(25, 5, 25, 5)
for i in (0, 1):
  # vol_cutoff_slider.children()[i].setAlignment(PyQt5.QtCore.Qt.AlignCenter)
  vol_cutoff_slider.children()[i].setFixedWidth(len(str(int(regions[-1]['vol']))) * 20)

def vol_cutoff_update():
  global LOW_VOLUME_CUTOFF, HIGH_VOLUME_CUTOFF, reconstructed_cells
  reconstructed_cells.fill(0)
  LOW_VOLUME_CUTOFF, HIGH_VOLUME_CUTOFF = vol_cutoff_slider.value()
  for region in regions:
    if LOW_VOLUME_CUTOFF <= region['vol'] <= HIGH_VOLUME_CUTOFF:
      minz, miny, minx, maxz, maxy, maxx = region['bbox']
      segmented_cell = region['image'] * denoised[minz:maxz, miny:maxy, minx:maxx]
      segmented_cell = segmented_cell / (segmented_cell.max() - segmented_cell.min())
      reconstructed_cells[minz:maxz, miny:maxy, minx:maxx] += segmented_cell
  minz, miny, minx, maxz, maxy, maxx = regions[n_region]['bbox']
  viewer.layers[0].data = reconstructed_cells

vol_cutoff_slider.valueChanged.connect(vol_cutoff_update)


@magicgui(
  call_button='Export Cells',
  output_option=dict(choices=['both', '3d', 'mip']),
  segment_type=dict(choices=['both', 'segmented', 'unsegmented'])
)
def export_cells(
  output_option,
  segment_type
):
  global OUTPUT_OPTION, SEGMENT_TYPE, reconstructed_cells
  OUTPUT_OPTION, SEGMENT_TYPE = output_option, segment_type
  ac.export_cells(CONFOCAL_TISSUE_IMAGE, LOW_VOLUME_CUTOFF,
                  HIGH_VOLUME_CUTOFF, OUTPUT_OPTION, denoised,
                  regions, None, SEGMENT_TYPE, NAME_ROI, roi_polygon)


viewer.window.add_dock_widget(vol_cutoff_slider, name='Volume Cutoff', area='bottom')

viewer.window._dock_widgets['Volume Cutoff'].setFixedHeight(80)
viewer.window.add_dock_widget(export_cells, name='Export Cells')
vol_cutoff_update()

In [ ]:
params = {#'CONTRAST_PTILES': CONTRAST_PTILES,
          'REF_IMAGE': REF_IMAGE,
          'CLIP_LIMIT': CLIP_LIMIT,
          'LOW_THRESH': LOW_THRESH,
          'HIGH_THRESH': HIGH_THRESH,
          'SELECT_ROI': SELECT_ROI,
          'NAME_ROI': NAME_ROI,
          'PRE_LOW_VOLUME_CUTOFF': PRE_LOW_VOLUME_CUTOFF,
          'LOW_VOLUME_CUTOFF': LOW_VOLUME_CUTOFF,  # filter noise/artifacts
          'HIGH_VOLUME_CUTOFF': HIGH_VOLUME_CUTOFF,  # filter cell clusters
          'OUTPUT_TYPE': SEGMENT_TYPE
}

DIR = getcwd() + '/Autocropped/'
IMAGE_NAME = '.'.join(path.basename(TISSUE_IMAGE).split('.')[:-1])
OUT_DIR = DIR + IMAGE_NAME + \
        f'{"" if NAME_ROI == "" else "-" + str(NAME_ROI)}/'
with open(OUT_DIR + '.params.json', 'w') as out:
    json.dump(params, out)


out = np.array([ALL_PT_ESTIMATES, FINAL_PT_ESTIMATES])
np.save(OUT_DIR + '.somas_estimates.npy', out)